In [15]:
import itertools
import math
import numpy as np


**WORDLE BEST GUESS CALCULATOR**

Step 1: Create functions that simulate the wordle game

- checked checks whether the position of the letter that is being checked in the answer word has been considered before. (prevents multiple green for one letter)
- iterates through the 5 letter word guessed one letter at a time comparing them to the answer
- returns an output string representing the word

In [22]:
def find(my_list, value):
    for index, element in enumerate(my_list):
        if element == value:
            return index
    return -1

In [26]:
def check_word(guess, answer):
    result = ['N'] * len(guess)  # Default all to 'N'
    answer_list = list(answer)
    
    # First pass: Check for correct positions (greens)
    for i, letter in enumerate(guess):
        if letter == answer_list[i]:
            result[i] = 'G'
            answer_list[i] = None  # Mark this letter as used
    
    # Second pass: Check for correct letters in wrong positions (yellows)
    for i, letter in enumerate(guess):
        if result[i] == 'N' and letter in answer_list:
            result[i] = 'Y'
            answer_list[answer_list.index(letter)] = None  # Mark this letter as used
    
    return "".join(result)

In [70]:
def create_hash_map():
    hash_map = {}
    for i in wordlist:
        for j in answerlist:
            hash_map[i + ", " + j] = check_word(i, j)
    return hash_map

In [72]:
word_map = create_hash_map()

- wordfile : file containing list of all valid words in a wordle game
- wordlist : wordfile content unpacked into a python list
- answerfile : file containing list of all answer words in a wordle game
- answerlist : answerfile content unpacked into a python list

In [11]:
wordfile = open(r'C:\Users\ASUS\Downloads\wordlist.txt', 'r')

In [13]:
wordlist = wordfile.read().split()

In [16]:
answerfile = open(r'C:\Users\ASUS\Downloads\solutionlist.txt', 'r')

In [18]:
answerlist = answerfile.read().split()

In [20]:
len(answerlist)

2309

In [22]:
len(wordlist)

14855

- <b> generate_permutations </b> : creates a list of all permutations of the specified characters
- <b> permutations </b> : list of all permutations possible in a wordle game
- <b> permutations_dict </b> : used for calculating probabilities of each permutation occuring for a specific word

In [24]:
def generate_permutations(characters):
    permutations = [''.join(p) for p in itertools.product(characters, repeat=5)]
    return permutations

In [22]:
permutations = generate_permutations(['G', 'N', 'Y']);

In [24]:
permutation_dict = {perm : 0 for perm in permutations}

<b> new_list </b> : used to update the list of words to a newer list based upon the result of a guess

In [190]:
def new_list(guess, permutation, list):
    newlist = []
    for answer in list:
        if check_word(guess, answer) == permutation:
            newlist.append(answer)
    return newlist

<b> find_probability_dict </b> : creates a dictionary of probabilities of each permutation for a specified guess

In [126]:
def find_probability_dict(guess, answerlist):
    probability_dict = {}
    for answer in answerlist:
        perm = word_map[guess + ", " + answer]
        probability_dict[perm] = probability_dict.get(perm, 0) + (1 / len(answerlist))
    return probability_dict

<b> calculate_entropy </b> : calculates the entropy for a word based on the probability dictionary generated by find_probability_dict

In [48]:
def calculate_entropy(probability_dict):
    entropy = 0
    for probability in probability_dict.values():
        entropy += -1 * (probability * math.log(probability, 2))
    return entropy

In [128]:
#Creating a copy to not alter any information within wordlist
remaining_words_list = wordlist.copy()

In [130]:
#Calculate the entropy for each word in the list
entropy_dict = {}
for word in remaining_words_list:
            probdict = find_probability_dict(word, answerlist)
            entropy = calculate_entropy(probdict)
            entropy_dict[word] = entropy

In [134]:
#Sort the dictionary based on entropy values
sorted_entropy_dict = dict(sorted(entropy_dict.items(), key = lambda item: item[1], reverse = True))

In [102]:
item_list = list(sorted_entropy_dict.items())
entropy_dict_two_steps = sorted_entropy_dict.copy()
word_list_copy = wordlist.copy()
answer_list_copy = answerlist.copy()

for i in range(0, 200):
    prob_dict1 = find_probability_dict(itemlist[i][0], answerlist)
    entropy_dictionary = permutation_dict.copy()
    for j in entropy_dictionary.keys():
        remaining_words_list = new_list(itemlist[i][0], j, answer_list_copy)
        max_entropy = 0
        for word in word_list_copy:
            prob_dict2 = find_probability_dict(word, remaining_words_list)
            entropy = calculate_entropy(prob_dict2)
            max_entropy = max(max_entropy, entropy)
        entropy_dictionary[j] += max_entropy * prob_dict1.get(j, 0)

    avg_entropy = sum(list(entropy_dictionary.values()))
    print("WORD : " + itemlist[i][0] + " ENTROPY : " + str(avg_entropy))
    entropy_dict_two_steps[itemlist[i][0]] += avg_entropy
    

WORD : tarse ENTROPY : 4.1068145555471665
WORD : tiare ENTROPY : 4.076608858095578
WORD : soare ENTROPY : 4.107623263495029
WORD : roate ENTROPY : 4.110602875896228
WORD : raise ENTROPY : 4.076821629156655
WORD : reast ENTROPY : 4.151920545448849
WORD : raile ENTROPY : 4.125283838598162
WORD : slate ENTROPY : 4.183570042068703
WORD : salet ENTROPY : 4.182314795562495
WORD : crate ENTROPY : 4.179291376808235
WORD : irate ENTROPY : 4.134813176196052
WORD : trace ENTROPY : 4.187590457766297
WORD : sater ENTROPY : 4.1853758617617425
WORD : arise ENTROPY : 4.115017193445619
WORD : orate ENTROPY : 4.137798593467071
WORD : stare ENTROPY : 4.182436100533191
WORD : carte ENTROPY : 4.204828670076519
WORD : raine ENTROPY : 4.187660639047795
WORD : ranse ENTROPY : 4.214081872134109
WORD : caret ENTROPY : 4.205392633582906
WORD : ariel ENTROPY : 4.1607654580817455
WORD : taler ENTROPY : 4.160382068927504
WORD : carle ENTROPY : 4.24078259902201
WORD : slane ENTROPY : 4.268327443048271
WORD : snare E

In [108]:
sorted_entropy_dict_two_steps = dict(sorted(entropy_dict_two_steps.items(), key = lambda item: item[1], reverse = True))
sorted_entropy_dict_two_steps

{'tarse': 10.055789065502687,
 'slate': 10.039389286178217,
 'slane': 10.037284764807037,
 'reast': 10.01965856629241,
 'salet': 10.018337577654979,
 'trace': 10.018019565846053,
 'crate': 10.014507359441517,
 'carle': 10.010147005753758,
 'sater': 10.00980946500571,
 'tiare': 10.006563080409087,
 'roast': 10.004724796339833,
 'torse': 10.001672954831172,
 'carte': 9.9999360049904,
 'carse': 9.998114992994116,
 'toile': 9.997879201173951,
 'trone': 9.99586073534721,
 'roate': 9.995459189628235,
 'soare': 9.992826007787786,
 'ranse': 9.992385221882884,
 'raile': 9.99043766763943,
 'stare': 9.98932472278296,
 'caret': 9.983599569512458,
 'crane': 9.979400410252289,
 'least': 9.977343724283344,
 'stale': 9.977315314563764,
 'carne': 9.975176739142272,
 'slart': 9.975025985194538,
 'raine': 9.973853994838336,
 'sacre': 9.973526570852595,
 'snare': 9.97087851090006,
 'trice': 9.970364214037517,
 'liane': 9.970170539739147,
 'stole': 9.969438516031188,
 'riant': 9.969105689451489,
 'trine': 

In [70]:
def get_next_guess(remaining_words_list, answer_list):
    max = 0
    maxword = "trace"
    permutation = input("Enter the result of the guess: ")
    answer_list = find_word(maxword, permutation, answer_list)
    while permutation != "GGGGG" :
        if len(answer_list) == 1:
            return answer_list[0]
        for word in remaining_words_list:
            probdict = find_probability_dict(word, answer_list)
            entropy = calculate_entropy(probdict)
            if entropy > max:
                max = entropy
                maxword = word
        print("MAXWORD : ", maxword)
        print(len(remaining_words_list))
        print(len(answer_list))
        permutation = input("Enter the result of the guess: ")
        answer_list = find_word(maxword, permutation, answer_list)
        maxword = answer_list[0]
        max = 0

Issue im currently facing:
idk how to optimize this
if i guess for all remaining words it doesnt seem to work well
but if i guess for all ansswer list words it works better but seems to still be sub optimal


In [96]:
def next_guess_initial_step(remaining_words_list, answer_list):
    max1 = 0
    maxword = ""
    for word in remaining_words_list:
            probdict = find_probability_dict(word, answer_list)
            entropy = calculate_entropy(probdict)
            if entropy > max1:
                max1 = entropy
                maxword = word
    return maxword

In [144]:
def next_guess_final_step(remaining_words_list, answer_list, original_answer_list):
    max1 = 0
    maxword = ""
    for word in remaining_words_list:
            probdict = find_probability_dict(word, answer_list)
            entropy = calculate_entropy(probdict)
            if word not in answer_list:
                entropy *= 0.3
            if entropy > max1:
                max1 = entropy
                maxword = word
    return maxword

In [227]:
def get_next_guess_automated(remaining_words_list, answer_list, answer, starting_word):
    max = 0
    maxword = starting_word
    original_answer_list = answer_list.copy()
    answer_list = new_list(maxword, check_word(maxword, answer) , answer_list)
    counter = 1
    while check_word(maxword, answer) != "GGGGG" :
        if len(answer_list) == 1:
            return counter + 1
        if len(answer_list) >= 4:
            maxword = next_guess_initial_step(remaining_words_list, answer_list)
        else:
            maxword = next_guess_final_step(remaining_words_list, answer_list, original_answer_list)
        answer_list = new_list(maxword, check_word(maxword, answer), answer_list)
        max = 0
        counter += 1
        
    return counter

In [15]:
get_next_guess_automated(a, b, "match","hatch")

3

In [221]:
a = wordlist.copy()
b = answerlist.copy()

In [223]:
get_next_guess_automated(a,b,"awake")

4

In [27]:
def store_word_counts(wordlist, answerlist, starting_word):
    hash_map = {}
    for answer in answerlist:
        counter = get_next_guess_automated(wordlist, answerlist, answer, starting_word)
        hash_map[answer] = counter
    return hash_map

In [29]:
def get_word_counts(wordlist, answerlist, starting_word):
    counter = 0
    for answer in answerlist:
        counter += get_next_guess_automated(wordlist, answerlist, answer, starting_word)
    return (counter/ len(answerlist))

In [33]:
for i in ["tarse", "salet", "slate", "reast", "trace", "crate"]:
    count = get_word_counts(wordlist, answerlist, i)
    print(i, " : ", count)

tarse  :  3.457773928107406


KeyboardInterrupt: 

In [250]:
len(number_of_steps_list)

2309

In [286]:
a = wordlist.copy()

In [302]:
b = answerlist.copy()

In [9]:
get_next_guess(a, b)

Enter the result of the guess:  NNYNY


MAXWORD :  kneel
14855
48


Enter the result of the guess:  NNNGG


MAXWORD :  baghs
14855
5


Enter the result of the guess:  NGNYN


'hazel'

In [ ]:
while True:
    inp = input()
    remaining_words_list = find_word("guess", inp, remaining_words_list)
    print(len(remaining_words_list
    if len(remaining_words_list) == 1:
        break


In [1]:
import dill
dill.load_session('notebook_env.db')

In [247]:
dill.dump_session('notebook_env.db')